Подготовим датасет к работе

In [5]:
import pandas as pd
import datetime

us = pd.read_csv('users.csv', sep='\t', encoding = 'koi8_r')
us.columns = ['user_id','email','geo']
us.user_id = us.user_id.apply(lambda x: x.lower())

log = pd.read_csv('log.csv', header = None)
log.columns = ['user_id', 'time', 'bet', 'win']
log = log[log.user_id != '#error']  

# log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])  
# log.time = log.time.apply(lambda x:str(x).replace("[",""))
# log.time = pd.to_datetime(log.time)

# log['bet'] = log['bet'].fillna(0)
# log['win'] = log['win'].fillna(0)
# log['net'] = log['win'] - log['bet']

# log = pd.merge(log, us, on='user_id')

,user_id,email,geo
0,user_943,Accumanst@gmail.com,Ижевск
1,user_908,Advismowr@mail.ru,Ижевск
2,user_962,Anachso@ukr.net,Краснодар
3,user_973,Antecia@inbox.ru,Пермь
4,user_902,Balliaryva@ukr.net,NaN
...,...,...,...
95,user_959,UpdatesCurious@yahoo.com,Тюмень
96,user_901,V2artierso@mail.ru,Арзангелтск
97,user_970,Vashoterlo@bk.ru,Воронеж
98,user_965,Visuareda@yahoo.com,Воронеж


,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,NaN
996,#error,NaN,10054.0,29265.0
997,#error,NaN,10454.0,NaN
998,#error,NaN,1000.0,NaN
